In [ ]:
# change this to whatever works to you
# you should be in the directory of the notebook
%cd pack_tutorial/   

In [65]:
from IPython.display import display, Markdown
from pathlib import Path

readme = Path("./readme.md").absolute()

with readme.open("r") as f:
    display(Markdown(f.read()))

# Compressing X-ray tomography volumes with multiple semantic segmentations (voxel-wise categorical values)

## Install

Install a minimal conda env for the pack_volumes.ipynb example.

My conda version:

`conda -V`

```
conda 4.9.0
```
### Create a conda env

Sequence of commands that I did:

```
conda create --name packenv python=3.8
conda activate packenv
conda install -c conda-forge basictools lxml pytables ipykernel
```
### `pymicro`

`pymicro` is not on conda, so let's install it with pip

```
~/.conda/envs/packenv/bin/pip install git+https://github.com/heprom/pymicro.git
```

You might have to change the path to get the correct pip (in your conda env). 

You should look for the `pip` file at somewhere like `/path/to/my/env/bin/`.

### `jupyter lab` (or `jupyter notebook` if you prefer)

#### option 1

If you want to install jupyter notebook/lab in the same env:

```
conda install -c conda-forge jupyterlab
```

#### option 2

If you already have a jupyter notebook/lab installed, you can create a kernel in env you just created (`packenv`) and use it from there:

```
ipython kernel install --user --name=packenv
```

Restart your jupyter notebook/lab sever (or launch another one with a custom port).
You will see the kernel `packenv` available.

Then:

```
conda deactivate
```

Have fun (:

## Env

See the files:
 - `packenv-hist.yml`: from `conda env export --from-history`
 - `packenv.yml`: from `conda env export`
 
## Data

The files used in this example can be downloaded in the link below. 

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.4587757.svg)](https://doi.org/10.5281/zenodo.4587757)


Extract them to a folder `raws` in the same directory of the tutorial. 

These are mock volumes from a glass fiber-reinforced polyamide 66 (`pa66`) image.

---

author: [joaopcbertoldo](joaopcbertoldo.github.io)

---

# Todos

 - reload data after writing it
 - plot it
 - get the last bit of the tutorial and put it with the published data to show how to open a sample data


In [3]:
from copy import copy
import os

import numpy as np
from matplotlib import pyplot as plt
import tables
import pymicro
from pymicro.file.file_utils import HST_read
from pymicro.core.samples import SampleData

# Loading the data

In [4]:
data_dir = Path("./raws").absolute()

In [5]:
sorted(os.listdir(data_dir))

['.ipynb_checkpoints',
 'crack.prediction.raw',
 'crack.prediction.raw.info',
 'crack.raw',
 'crack.raw.info',
 'pa66.ground_truth.raw',
 'pa66.ground_truth.raw.info',
 'pa66.raw',
 'pa66.raw.info',
 'pa66.test.error_volume.raw',
 'pa66.test.error_volume.raw.info',
 'pa66.test.prediction.raw',
 'pa66.test.prediction.raw.info']

In [6]:
pa66 = dict(
    data="pa66.raw",
    ground_truth="pa66.ground_truth.raw",
    prediction="pa66.test.prediction.raw",
    error="pa66.test.error_volume.raw",
)

crack = dict(
    data="crack.raw",
    prediction="crack.prediction.raw",
)


# transform the filenames in Path objects - it's handier (8
pa66 = {
    key: data_dir / fname 
    for key, fname in pa66.items()
}

crack = {
    key: data_dir / fname 
    for key, fname in crack.items()
}

File sizes

In [7]:
def show_file_sizes(files_dic):
    for key, file in files_dic.items():
        print(f"{key}: {file.stat().st_size / 1024**3:.2f} GB")

In [8]:
show_file_sizes(pa66)

data: 0.06 GB
ground_truth: 0.06 GB
prediction: 0.02 GB
error: 0.02 GB


Notice that 2 of the files have 1900 z-slices (the bigger ones), and 2 have 300 z-slices (the test set, a subvolume of the others).

In [9]:
def show_info_files(files_dic):

    for key, file in files_dic.items():

        info_file = file.with_suffix(".raw.info")
        print(f"{info_file.name}")

        with info_file.open("r") as f:
            print(f.read(), "\n")

In [10]:
show_info_files(pa66)

pa66.raw.info
! PyHST_SLAVE VOLUME INFO FILE
NUM_X = 1300
NUM_Y = 1040
NUM_Z =   50
DATA_TYPE = uint8
 

pa66.ground_truth.raw.info
! PyHST_SLAVE VOLUME INFO FILE
NUM_X = 1300
NUM_Y = 1040
NUM_Z =   50
DATA_TYPE = uint8
 

pa66.test.prediction.raw.info
! PyHST_SLAVE VOLUME INFO FILE
NUM_X = 1300
NUM_Y = 1040
NUM_Z =   15
DATA_TYPE = uint8
 

pa66.test.error_volume.raw.info
! PyHST_SLAVE VOLUME INFO FILE
NUM_X = 1300
NUM_Y = 1040
NUM_Z =   15
DATA_TYPE = uint8
 



In [12]:
# I am doing this automatically because all my files are 
# conveniently in the same data type, but you could 
# do this manually as well

pa66_volumes = {
    key: HST_read(
        str(file),
        autoparse_filename=False,
        data_type=np.uint8,
    )
    for key, file in pa66.items()  # pa66 here
}

In [13]:
def check_volumes(volumes_dic):
    for key, arr in volumes_dic.items():
        print(f"{key=} \n{arr.dtype=} \n{arr.shape=}\n")

In [14]:
check_volumes(pa66_volumes)

key='data' 
arr.dtype=dtype('uint8') 
arr.shape=(1300, 1040, 50)

key='ground_truth' 
arr.dtype=dtype('uint8') 
arr.shape=(1300, 1040, 50)

key='prediction' 
arr.dtype=dtype('uint8') 
arr.shape=(1300, 1040, 15)

key='error' 
arr.dtype=dtype('uint8') 
arr.shape=(1300, 1040, 15)



# Building the `SampleData` object

This is where we put things in an `.h5`

In [15]:
# the compression algorithm
ft = tables.Filters(
    complevel=9, 
    complib='zlib', 
    shuffle=True, 
    bitshuffle=False,
)

## `SampleData`

In [16]:
bibtex_citation = "TO BE UPDATED"

sample_description=""\
"Glass fiber-reinforced Poly Amide 66 3D X-ray "\
"tomography dataset for deep learning segmentation.\n"\
f"If you use this volume, please cite us: {bibtex_citation}"

sample = SampleData(
    filename="pa66_volumes",  # for "PolyAmide66"
    sample_name="pa66_volumes", 
    sample_description=sample_description,
    filters=ft,
    replace=True,
)

## all sets image (train, val, test)

data

In [17]:
sample.add_image_from_field(
    location="/",
    imagename="pa66",
    fieldname="data",
    field_array=pa66_volumes["data"],
    filters=ft,
    origin=np.array([0, 0, 0]),
    replace=True,
)

In [18]:
sample.set_description(
    "Annotated dataset, used for training, validation, and test.", 
    node="/pa66"
)

In [19]:
sample.set_description(
    "Data (gray level image stack)", 
    node="/pa66/data",
)

In [20]:
sample

Dataset Content Index :
------------------------:
index printed with max depth `3` and under local root `/`

	 Name : pa66                                      H5_Path : /pa66 	
	 Name : data                                      H5_Path : /pa66/data 	

****** DATA SET CONTENT ******
 -- File: pa66_volumes.h5
 -- Size:    55.050 Mb
 -- Data Model Class: SampleData

 Group /
 -- Parent Group : /
 -- Group attributes : 
	 description : Glass fiber-reinforced Poly Amide 66 3D X-ray tomography dataset for deep learning segmentation.
If you use this volume, please cite us: TO BE UPDATED
	 sample_name : pa66_volumes
 -- Childrens : Index, pa66, 
----------------
************************************************
 Group pa66
 -- Parent Group : /
 -- Group attributes : 
	 Field_index : ['data']
	 description : Annotated dataset, used for training, validation, and test.
	 dimension : [1300 1040   50]
	 empty : False
	 group_type : 3DImage
	 nodes_dimension : [1301 1041   51]
	 nodes_dimension_xdmf

##### ground truth

In [21]:
sample.add_field(
    gridname="/pa66",
    fieldname="ground_truth",
    array=pa66_volumes["ground_truth"],
    filters=ft,
    replace=True,
)

/pa66/ground_truth (CArray(50, 1040, 1300), shuffle, zlib(9)) 'ground_truth'
  atom := UInt8Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := (1, 100, 1300)

In [22]:
sample.set_description(
    "Ground truth prediction. "\
    "Values are 0, 1, and 2, respectively corresponding "
    "to the phases matrix, fiber, and porosity.", 
    node="/pa66/ground_truth",
)

## test set image

prediction

notice that the origin is different

In [23]:
sample.add_image_from_field(
    location="/",
    imagename="pa66_test",
    fieldname="prediction",
    field_array=pa66_volumes["prediction"],
    filters=ft,
    origin=np.array([0, 0, 7]),  # todo change me to 1600
    replace=True,
)

In [24]:
sample.set_description(
    "Test set (last 300 layers of the volume).", 
    node="/pa66_test"
)

In [25]:
sample.set_description(
    "Prediction generated by a 2d u-net model.", 
    node="/pa66_test/prediction",
)

errors

In [26]:
sample.add_field(
    gridname="/pa66_test",
    fieldname="error",
    array=pa66_volumes["error"],
    filters=ft,
    replace=True,
)

/pa66_test/error (CArray(15, 1040, 1300), shuffle, zlib(9)) 'error'
  atom := UInt8Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := (1, 100, 1300)

In [27]:
sample.set_description(
    "Disagreement between the ground truth and "
    "the model's prediction on the test set: "
    "1 means 'incorrect', 0 means 'correct'.", 
    node="/pa66_test/error",
)

In [28]:
sample

Dataset Content Index :
------------------------:
index printed with max depth `3` and under local root `/`

	 Name : pa66                                      H5_Path : /pa66 	
	 Name : data                                      H5_Path : /pa66/data 	
	 Name : ground_truth                              H5_Path : /pa66/ground_truth 	
	 Name : pa66_test                                 H5_Path : /pa66_test 	
	 Name : prediction                                H5_Path : /pa66_test/prediction 	
	 Name : error                                     H5_Path : /pa66_test/error 	

****** DATA SET CONTENT ******
 -- File: pa66_volumes.h5
 -- Size:    60.200 Mb
 -- Data Model Class: SampleData

 Group /
 -- Parent Group : /
 -- Group attributes : 
	 description : Glass fiber-reinforced Poly Amide 66 3D X-ray tomography dataset for deep learning segmentation.
If you use this volume, please cite us: TO BE UPDATED
	 sample_name : pa66_volumes
 -- Childrens : Index, pa66, pa66_test, 
----------------
****

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.4587757.svg)](https://doi.org/10.5281/zenodo.4587757)# crack

adding an image of another volume

In [29]:
show_file_sizes(crack)

data: 0.14 GB
prediction: 0.14 GB


In [30]:
show_info_files(crack)

crack.raw.info
! PyHST_SLAVE VOLUME INFO FILE
NUM_X = 1579
NUM_Y = 1845
NUM_Z =   50
DATA_TYPE = uint8
 

crack.prediction.raw.info
! PyHST_SLAVE VOLUME INFO FILE
NUM_X = 1579
NUM_Y = 1845
NUM_Z =   50
DATA_TYPE = uint8
 



In [31]:
crack_volumes = {
    key: HST_read(
        str(file),
        autoparse_filename=False,
        data_type=np.uint8,
        zrange=range(50),  # todo: removeme
    )
    for key, file in crack.items()  # crack here
}

In [32]:
check_volumes(crack_volumes)

key='data' 
arr.dtype=dtype('uint8') 
arr.shape=(1579, 1845, 50)

key='prediction' 
arr.dtype=dtype('uint8') 
arr.shape=(1579, 1845, 50)



##### data

In [33]:
sample.add_image_from_field(
    location="/",
    imagename="crack",
    fieldname="crack_data",
    indexname="crack",
    field_array=crack_volumes["data"],
    filters=ft,
    origin=np.array([0, 0, 0]),
    replace=True,
)

In [34]:
sample.set_description(
    "Unannotated volume of glass-reinforced polyamide 66 submited to "\
    "a biaxial test, containg a crack issued from the experiment.", 
    node="/crack"
)

In [35]:
sample.set_description(
    "Data (gray level image stack).", 
    node="/crack/crack_data",
)

##### prediction

In [36]:
sample.add_field(
    gridname="/crack",
    fieldname="crack_prediction",
    array=crack_volumes["prediction"],
    filters=ft,
    replace=True,
)

/crack/crack_prediction (CArray(50, 1845, 1579), shuffle, zlib(9)) 'crack_prediction'
  atom := UInt8Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := (1, 166, 1579)

In [37]:
sample.set_description(
    "Segmentation predicted by the model trained on the other dataset.", 
    node="/crack/crack_prediction",
)

## metadata

In [38]:
voxel_size = np.array([1.3, 1.3, 1.3])

sample.set_voxel_size("pa66", voxel_size)
sample.set_voxel_size("pa66_test", voxel_size)
sample.set_voxel_size("crack", voxel_size)

In [39]:
class_mapping = dict(
    matrix=0,
    fiber=1,
    porosity=2,
)

error_mapping = dict(
    correct=0,
    error=1,
)

data_split = dict(
    train=dict(
        x_range=[0, 1300],
        y_range=[0, 1040],
        z_range=[0, 1300],
    ),
    val=dict(
        x_range=[0, 1300],
        y_range=[0, 1040],
        z_range=[1078, 1206],
    ),
    test=dict(
        x_range=[0, 1300],
        y_range=[0, 1040],
        z_range=[1300, 1600],
    ),
)

In [40]:
sample.add_attributes(
    dict(data_split=copy(data_split)), 
    "/pa66",
)

In [41]:
sample.add_attributes(
    dict(class_mapping=copy(class_mapping)),
    "/pa66/ground_truth"
)

In [42]:
sample.add_attributes(
    dict(class_mapping=copy(class_mapping)), 
    "/pa66_test/prediction",
)

In [43]:
sample.add_attributes(
    dict(error_mapping=copy(error_mapping)), 
    "/pa66_test/prediction",
)

In [44]:
sample.add_attributes(
    dict(class_mapping=copy(class_mapping)), 
    "/crack/crack_prediction",
)

In [45]:
sample.sync()

In [46]:
del sample

# Test 

## load 

In [47]:
sample

NameError: name 'sample' is not defined

:point-up: that should give a `NameError`

In [48]:
reloaded_sample = SampleData("pa66_volumes")

check that all the info is back there

In [50]:
reloaded_sample

Dataset Content Index :
------------------------:
index printed with max depth `3` and under local root `/`

	 Name : crack                                     H5_Path : /crack 	
	 Name : crack_data                                H5_Path : /crack/crack_data 	
	 Name : crack_prediction                          H5_Path : /crack/crack_prediction 	
	 Name : data                                      H5_Path : /pa66/data 	
	 Name : error                                     H5_Path : /pa66_test/error 	
	 Name : ground_truth                              H5_Path : /pa66/ground_truth 	
	 Name : pa66                                      H5_Path : /pa66 	
	 Name : pa66_test                                 H5_Path : /pa66_test 	
	 Name : prediction                                H5_Path : /pa66_test/prediction 	

****** DATA SET CONTENT ******
 -- File: pa66_volumes.h5
 -- Size:   165.840 Mb
 -- Data Model Class: SampleData

 Group /
 -- Parent Group : /
 -- Group attributes : 
	 description : Glas

## plot